# Catnip

In [1]:
# Dependencies and configuration
from src.config import settings, setup_dirs

izutsumiPaths, notIzutsumiPaths = setup_dirs()

## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel


_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
from src.preprocess.headExtraction import anime_extraction_recursive


valid_exts = {".jpg", ".jpeg", ".png"}
panel_paths = sorted(
    [p for p in settings.paths.panels_dir.iterdir() if p.suffix.lower() in valid_exts]
)
num_crops = anime_extraction_recursive()

print(f"Extracted {num_crops} faces")

## Catnip core

### MobileNetV2 Model

In [ ]:
from src.recognition.embeddingModel import compute_embeddings, build_model, load_embeddings


# Build fresh model without loading weights
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=False)
compute_embeddings(embed_model, settings.paths.crops_dir, settings.params.img_size)


# Build embedding model (loads saved weights if available)
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=True)
embs, crop_paths = load_embeddings(settings.paths.embs_dir, settings.paths.crops_dir)


In [ ]:
from src.recognition.query import izutsumi_query, izutsuminess_rank


crop, index, score, thre = izutsumi_query(settings.paths.embs_dir,
                                        settings.paths.crops_dir,
                                        settings.params.img_size, 
                                        embed_model, 
                                        izutsumiPaths,
                                        notIzutsumiPaths,
                                        similarity_threshold=-1,
                                        alpha=0.5,
                                        mode='max')

index_log = izutsuminess_rank(settings.paths.embs_dir, settings.paths.crops_dir, embed_model, izutsumiPaths, notIzutsumiPaths)

### YOLOv8 fine-tuning

In [ ]:
from src.training.preparation import prepare_data

prepare_data(izutsumiPaths, notIzutsumiPaths)

In [ ]:
# For Apple Silicon with MPS enabled on izutsumiTraining.yaml
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [2]:
from src.training.training import train_model, build_model

model = build_model(settings.paths.model_dir/'yolov8_izutsumi_finetuned.pt')

train_model(model,
            epochs=50, 
            imgsz=settings.params.img_size, 
            batch=16, 
            lr0=1e-3, 
            freeze=8,
            workers=8,
            resume=False)

New https://pypi.org/project/ultralytics/8.3.214 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.6.0 CPU (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config/izutsumiTraining.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=8, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=data/models/yolov8_izutsumi_finetuned.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp14, nbs=64, nms=False, op

In [3]:
# Evaluate
metrics = model.val()
print(metrics)

# Predict on unseen images
model.predict(
    source="data/processed/izutsumiTraining/val/images",
    save=True,
    conf=0.5
)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.6.0 CPU (Apple M3)
YOLOv8x6 summary (fused): 147 layers, 97,282,200 parameters, 0 gradients, 260.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 60.2±23.1 MB/s, size: 12.6 KB)
val: Scanning /Users/rifusaki/repos/catnip/data/processed/izutsumiTraining/val/labels.cache... 48 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 48/48 173.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 0.3it/s 8.7s5.3ss
                   all         48         48      0.892       0.95      0.979      0.979
              izutsumi         36         36       0.97        0.9      0.988      0.988
          not izutsumi         12         12      0.814          1       0.97       0.97
Speed: 0.0ms preprocess, 174.4ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /Users/rifusaki/repos/catnip/runs/izutsumi_finetune/exp142
ultralytics.utils.metrics.DetMetrics ob

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'izutsumi', 1: 'not izutsumi'}
 obb: None
 orig_img: array([[[  4,   2,   2],
         [  3,   1,   1],
         [  2,   0,   0],
         ...,
         [254, 254, 254],
         [190, 190, 190],
         [ 12,  12,  12]],
 
        [[  2,   0,   0],
         [  2,   0,   0],
         [  6,   4,   4],
         ...,
         [254, 254, 254],
         [159, 159, 159],
         [  4,   4,   4]],
 
        [[  1,   1,   1],
         [  1,   1,   1],
         [  5,   3,   3],
         ...,
         [253, 253, 253],
         [125, 125, 125],
         [  2,   2,   2]],
 
        ...,
 
        [[  2,   0,   0],
         [  5,   3,   3],
         [  9,   7,   7],
         ...,
         [234, 233, 235],
         [ 43,  42,  44],
         [  9,   8,  10]],
 
        [[  5,   3,   3],
         [  2,   0,   0],
         [119, 117, 117],
         ..

In [ ]:
model.save("data/models/yolov8_izutsumi_finetuned.pt")

## Outputting

In [ ]:
from src.output.output import save_similar_results, char_nearest_neighbor
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(range(len(index)), score)
plt.xlabel("Izutsuminess")
plt.ylabel("Ranked Results")
plt.title("Embed score")
plt.show()

In [ ]:
cutoff = 0

results = char_nearest_neighbor(crop, index[cutoff:], score[cutoff:], thre)

In [ ]:
save_similar_results(crop, index, settings.paths.output_dir, score)